In [21]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent, _time_distributed_dense
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

ImportError: cannot import name '_time_distributed_dense'

In [23]:
from keras.layers.recurrent import TimeD

ImportError: cannot import name '_time_distributed_dense'

In [2]:
from random import randint
from numpy import array
from numpy import argmax
 
# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(0, n_unique-1) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# prepare data for the LSTM
def get_pair(n_in, n_out, n_unique):
	# generate random sequence
	sequence_in = generate_sequence(n_in, n_unique)
	sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in-n_out)]
	# one hot encode
	X = one_hot_encode(sequence_in, n_unique)
	y = one_hot_encode(sequence_out, n_unique)
	# reshape as 3D
	X = X.reshape((1, X.shape[0], X.shape[1]))
	y = y.reshape((1, y.shape[0], y.shape[1]))
	return X,y
 
# generate random sequence
X, y = get_pair(5, 2, 50)
print(X.shape, y.shape)
print('X=%s, y=%s' % (one_hot_decode(X[0]), one_hot_decode(y[0])))

(1, 5, 50) (1, 5, 50)
X=[39, 21, 24, 25, 15], y=[39, 21, 0, 0, 0]


# Encoder Decoder without Attention

In [6]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2

model = Sequential()
model.add(LSTM(150, input_shape=(n_timesteps_in, n_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150, return_sequences = True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 150)               120600    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 5, 150)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 5, 150)            180600    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 50)             7550      
Total params: 308,750
Trainable params: 308,750
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
for epoch in range(5000):
    
    X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    model.fit(X,y, epochs=1, verbose=2)

Epoch 1/1
 - 1s - loss: 3.9074 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.9044 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.9042 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8800 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8806 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8633 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 3.8206 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8446 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7908 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7705 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7668 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7320 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7514 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6870 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6303 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5861 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5048 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4329 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.1907 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.9181 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.7424 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.6316 - acc: 0.600

Epoch 1/1
 - 0s - loss: 1.6425 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5914 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6040 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5803 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4600 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5509 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6162 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6370 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6435 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6316 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5892 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6365 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5259 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5063 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6794 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6014 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6352 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5892 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6680 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6131 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6206 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6652 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.6515 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6962 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6450 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7277 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5839 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5803 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6667 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5942 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5782 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5825 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5395 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5644 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5272 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5949 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6147 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6614 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6302 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6112 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5706 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5596 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5979 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5900 - acc: 0.6000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 1.5395 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5585 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6539 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5657 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4505 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5616 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5921 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4771 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5772 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5688 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5888 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6989 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7081 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5888 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5289 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4984 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5482 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5680 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6581 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6182 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5962 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6261 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.5365 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5267 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4070 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5106 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6450 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5768 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6365 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6089 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5831 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5728 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5128 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5623 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6155 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3529 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5855 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6154 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4743 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5244 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6326 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7001 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5161 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6173 - acc: 0.6000
Epoch 1/1
 - 0s - lo

...

Epoch 1/1
 - 0s - loss: 0.4610 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7746 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7075 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2692 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7589 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3089 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7620 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3951 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6012 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8693 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3291 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6606 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3137 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4379 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0017 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5973 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6707 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4890 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3969 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7950 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4414 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7137 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 1.1310 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.5675 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6495 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7876 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2627 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3586 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2187 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2498 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7157 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6422 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9723 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6966 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9325 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2742 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5240 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4966 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5113 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8593 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6635 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6169 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7330 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7340 - acc: 0.8000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.5079 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1139 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4720 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4519 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3395 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4532 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0253 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6469 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5235 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4392 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3990 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5917 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5594 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3873 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0811 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2484 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3748 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4581 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4705 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8872 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4684 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5794 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 0.5321 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5847 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6223 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5046 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6819 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6704 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1321 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4420 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5076 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3379 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8211 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6909 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4481 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5283 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8344 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4350 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4083 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1402 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5147 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8243 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2571 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1366 - acc: 1.0000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.7693 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4363 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7937 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4186 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5588 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4683 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6239 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3551 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4011 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5657 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3730 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7614 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4982 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0170 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3880 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6016 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3105 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8959 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4801 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5750 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4409 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9240 - acc: 0.8000
Epoch 1/1


In [12]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal

total, correct = 100,0

for _ in range(total):
    
    X,y= get_pair(n_timesteps_in,n_timesteps_out, n_features)
    yhat = model.predict(X,verbose=0)
    
    if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
        correct += 1
        
print('Accuracy: %.2f%%' % (float(correct)/float(total) * 100))

for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	print('Expected:', one_hot_decode(y[0]), 'Predicted', one_hot_decode(yhat[0]))

Accuracy: 25.00%
Expected: [2, 42, 0, 0, 0] Predicted [2, 2, 0, 0, 0]
Expected: [26, 6, 0, 0, 0] Predicted [26, 26, 0, 0, 0]
Expected: [33, 21, 0, 0, 0] Predicted [33, 33, 0, 0, 0]
Expected: [22, 2, 0, 0, 0] Predicted [22, 2, 0, 0, 0]
Expected: [46, 15, 0, 0, 0] Predicted [46, 46, 0, 0, 0]
Expected: [1, 22, 0, 0, 0] Predicted [1, 1, 0, 0, 0]
Expected: [35, 12, 0, 0, 0] Predicted [35, 35, 0, 0, 0]
Expected: [4, 44, 0, 0, 0] Predicted [4, 44, 0, 0, 0]
Expected: [25, 23, 0, 0, 0] Predicted [25, 0, 0, 0, 0]
Expected: [0, 30, 0, 0, 0] Predicted [0, 0, 0, 0, 0]


In [18]:
from attention_decoder import AttentionDecoder

model = Sequential()
model.add(LSTM(150, input_shape = (n_timesteps_in, n_features), return_sequences=True))
model.add(AttentionDecoder(150, n_features))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

ImportError: cannot import name 'AttentionDecoder'